In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.1.2'
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backpor

In [2]:
# For connection to Postgres
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-07-02 13:08:32--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.24MB/s    in 0.2s    

2021-07-02 13:08:32 (5.24 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
 from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("level_1_b").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

## Connect to s3 data

In [4]:
url="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
from pyspark import SparkFiles
spark.sparkContext.addFile(url)
spark_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Video_Games_v1_00.tsv.gz"), sep="\t", header=True)
spark_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

marketplace       - 2 letter country code of the marketplace where the review was written. <br>
customer_id       - Random identifier that can be used to aggregate reviews written by a single author.<br>
review_id         - The unique ID of the review.<br>
product_id        - The unique Product ID the review pertains to. In the multilingual dataset the reviews
                    for the same product in different countries can be grouped by the same product_id.<br>
product_parent    - Random identifier that can be used to aggregate reviews for the same product.<br>
product_title     - Title of the product.<br>
product_category  - Broad product category that can be used to group reviews 
                    (also used to group the dataset into coherent parts).<br>
star_rating       - The 1-5 star rating of the review.<br>
helpful_votes     - Number of helpful votes.<br>
total_votes       - Number of total votes the review received.<br>
vine              - Review was written as part of the Vine program.<br>
verified_purchase - The review is on a verified purchase.<br>
review_headline   - The title of the review.<br>
review_body       - The review text.<br>
review_date       - The date the review was written.

In [5]:
# Removed duplicate rows
spark_df = spark_df.dropDuplicates()
spark_df.count()

1785997

## Create datFrame to match the sql table schema

In [6]:
# Create dataframe for review_id_table
from pyspark.sql.functions import to_date

review_id_df = spark_df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")]).dropDuplicates(["review_id"])
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1004PYTPK6ELD|   38281029|B00004WHWF|      89143877| 2002-01-23|
|R1006LDKUHBWUK|   30949842|B00FRESY6G|     660321577| 2015-01-20|
|R100DAG2IK0FKR|   11334948|B000FRVCAA|     876011014| 2012-01-30|
|R101B69JLWW7AS|   12460262|B001B1W3HA|     431775211| 2013-03-16|
|R10292JZNM6DM6|   43092063|B00009WAUZ|     650007294| 2003-10-23|
| R102N5FRJ4BU2|   21435531|B00GMFKYJ4|     680106065| 2014-12-14|
|R104WH238ORTDT|   11213579|B00000IFRI|      14270449| 2013-07-04|
|R10571GO54Q7O3|   26879554|B00AAQRNQ8|     807565333| 2015-06-27|
|R105U7CTZNLWPE|    1355533|B0076HD2W8|     416708066| 2015-02-04|
|R106NTY9QICS5Q|   42329288|B00NKKPC1E|      74444111| 2015-01-12|
|R106PJYIP0RVAV|   36737011|B00008XKZZ|     999958929| 2012-07-27|
|R107YHOX9CYQWZ|   25972274|B00J48MUS4|     928566066| 2015-03

In [7]:
review_id_df.count()

1785997

In [8]:
# Create dataframe for products in sql 
# 1785997 with duplicates
# removed duplicates based on product_id to get 65792 records
products_df = spark_df.select(["product_id", "product_title"]).dropDuplicates(["product_id"])
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|0439394422|school house rock...|
|9861064222|Black Nintendo Wi...|
|B00000K1XN|        WWF Attitude|
|B00001OX40|   Neo Cherry Master|
|B00001QEQD|     USCF Chess - PC|
|B00001XDTO|        Yoda Stories|
|B00001ZWQ0|Legend of the Riv...|
|B0000296ZL|      Game Boy Color|
|B00002SFMS|     Boss Rally - PC|
|B00002SSYX|Ristar (Sega Game...|
|B00002STA7|    NFL Football '94|
|B00002STH3|           Extreme G|
|B00002STJC|                 MDK|
|B00002STWV|Virtual Open Tenn...|
|B00002SW1K|        World Heroes|
|B00002SW49|Lawnmower Man - S...|
|B000035XH1|RoboCop vs. Termi...|
|B000035XYM|Alien vs. Predato...|
|B000035Y3U|Top Gear 2 - Nint...|
|B000046S3Z|           Crystalis|
+----------+--------------------+
only showing top 20 rows



In [9]:
products_df.count()

65792

In [10]:
# Create dataframe for cusomers in sql
counts = spark_df.groupBy("customer_id").count()
customers_df = counts.select(["customer_id", "count"]).withColumnRenamed("count", "customer_count")
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|    5401088|             5|
|   37964777|            16|
|   20872523|             1|
|   44777937|             4|
|    2384511|             1|
|   41500251|             1|
|   34029693|             1|
|   40005173|             2|
|   12125016|             2|
|   21226328|             3|
|    6004768|             2|
|   14445274|             4|
|   41665035|             1|
|   16995171|             1|
|   25044130|             1|
|   11073960|             2|
|   38642776|             8|
|    6199838|             6|
|   19207466|             3|
|    1760318|             1|
+-----------+--------------+
only showing top 20 rows



In [11]:
# Create df for vine table
vine_df = spark_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"]).dropDuplicates(["review_id"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1004PYTPK6ELD|          5|            0|          0|   N|
|R1006LDKUHBWUK|          5|            0|          0|   N|
|R100DAG2IK0FKR|          2|            0|          1|   N|
|R101B69JLWW7AS|          5|            0|          0|   N|
|R10292JZNM6DM6|          3|            4|          5|   N|
| R102N5FRJ4BU2|          5|            0|          1|   N|
|R104WH238ORTDT|          4|            0|          0|   N|
|R10571GO54Q7O3|          5|            0|          0|   N|
|R105U7CTZNLWPE|          3|            0|          0|   N|
|R106NTY9QICS5Q|          5|            1|          1|   N|
|R106PJYIP0RVAV|          5|            0|          0|   N|
|R107YHOX9CYQWZ|          5|            0|          0|   N|
|R108IA3PLUFINS|          5|            0|          0|   N|
|R109OZ2ATS9SVB|          5|            

In [12]:
vine_df.count()

1785997



```
CREATE TABLE review_id_table (
  review_id TEXT PRIMARY KEY NOT NULL,
  customer_id INTEGER,
  product_id TEXT,
  product_parent INTEGER,
  review_date DATE -- this should be in the formate yyyy-mm-dd
);

-- This table will contain only unique values
CREATE TABLE products (
  product_id TEXT PRIMARY KEY NOT NULL UNIQUE,
  product_title TEXT
);

-- Customer table for first data set
CREATE TABLE customers (
  customer_id INT PRIMARY KEY NOT NULL UNIQUE,
  customer_count INT
);

-- vine table
CREATE TABLE vine_table (
  review_id TEXT PRIMARY KEY,
  star_rating INTEGER,
  helpful_votes INTEGER,
  total_votes INTEGER,
  vine TEXT
);
```



In [13]:
!pip install psycopg2-binary

     |████████████████████████████████| 3.4MB 24.2MB/s 


In [14]:
server = "cwrudb.******.us-east-2.rds.amazonaws.com"
database = "amzn_review"
port = "5432"
username = "root"
password = "****"

conn = f"postgresql://{username}:{password}@{server}:{port}/{database}"

from sqlalchemy import create_engine 
engine = create_engine(conn, echo=False)

In [16]:
# Review ID table, products table, cusomers table, vine table changed to pandas dataframe
pd_review_id_df = review_id_df.toPandas()
pd_products_df = products_df.toPandas()
pd_customers_df = customers_df.toPandas()
pd_vine_df = vine_df.toPandas()

In [17]:
# load the dataframe to postgresql aws rds database
pd_review_id_df.to_sql("review_id_table", con=engine, if_exists='append',index=False)
pd_products_df.to_sql("products", con=engine, if_exists="append", index=False)
pd_customers_df.to_sql("customers", con=engine, if_exists="append", index=False)
pd_vine_df.to_sql("vine_table", con=engine, if_exists="append", index=False)